In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from sqlalchemy import create_engine
#pip install numpy pandas scipy
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import seaborn as sns
# Load your dataset (replace 'your_dataset.csv' with the actual file path)
# load data
data = pd.read_csv(r'/content/engine_data.csv')




In [2]:
# Data Preprocessing & EDA

data.info()

data.head()

data.describe()
# calculate moments for each column

df = pd.DataFrame(data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19535 entries, 0 to 19534
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Engine rpm        19535 non-null  int64  
 1   Lub oil pressure  19535 non-null  float64
 2   Fuel pressure     19535 non-null  float64
 3   Coolant pressure  19535 non-null  float64
 4   lub oil temp      19535 non-null  float64
 5   Coolant temp      19535 non-null  float64
 6   Engine Condition  19535 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 1.0 MB


In [3]:
# Calculate moments for each column
numerical_cols = df.select_dtypes(include=np.number).columns

for col in numerical_cols:
    print(f"Column: {col}")
    print(f"1st Moment (Mean): {np.mean(df[col]):.2f}")
    print(f"1st Moment (Median): {np.median(df[col]):.2f}")
    print(f"1st Moment (Mode): {stats.mode(df[col]).mode:.2f}")
    print(f"2nd Moment (Variance): {np.var(df[col]):.2f}")
    print(f"2nd Moment (Standard deviation): {np.std(df[col]):.2f}")
    print(f"3rd Moment (Skewness): {stats.skew(df[col]):.2f}")
    print(f"4th Moment (Kurtosis): {stats.kurtosis(df[col]):.2f}\n")


Column: Engine rpm
1st Moment (Mean): 791.24
1st Moment (Median): 746.00
1st Moment (Mode): 620.00
2nd Moment (Variance): 71612.08
2nd Moment (Standard deviation): 267.60
3rd Moment (Skewness): 0.93
4th Moment (Kurtosis): 0.94

Column: Lub oil pressure
1st Moment (Mean): 3.30
1st Moment (Median): 3.16
1st Moment (Mode): 2.82
2nd Moment (Variance): 1.04
2nd Moment (Standard deviation): 1.02
3rd Moment (Skewness): 0.20
4th Moment (Kurtosis): -0.26

Column: Fuel pressure
1st Moment (Mean): 6.66
1st Moment (Median): 6.20
1st Moment (Mode): 4.60
2nd Moment (Variance): 7.62
2nd Moment (Standard deviation): 2.76
3rd Moment (Skewness): 1.22
4th Moment (Kurtosis): 2.45

Column: Coolant pressure
1st Moment (Mean): 2.34
1st Moment (Median): 2.17
1st Moment (Mode): 1.07
2nd Moment (Variance): 1.07
2nd Moment (Standard deviation): 1.04
3rd Moment (Skewness): 1.31
4th Moment (Kurtosis): 2.56

Column: lub oil temp
1st Moment (Mean): 77.64
1st Moment (Median): 76.82
1st Moment (Mode): 76.14
2nd Moment

In [4]:
# Separate features and target variable
X = data.drop(columns=['Engine Condition'])
y = data['Engine Condition']

# Define preprocessing steps
numeric_features = ['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure',
                    'lub oil temp', 'Coolant temp']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])
# Save the preprocessor (ColumnTransformer) pipeline
joblib.dump(preprocessor, 'preprocessor.pkl')

# Save the cleaned data using joblib
joblib.dump(data, 'cleaned_data.pkl')
print("Cleaned data saved successfully!")


Cleaned data saved successfully!


In [5]:
# Separate features and target variable
X = data.drop(columns=['Engine Condition'])
y = data['Engine Condition']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)


In [8]:
#automl
#TPOT classifier
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=17dc9d4921dc2bf7f42a16a9963cc4ff18d01648758fab6f93ea7cdd8f4bc088
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [6]:
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split


In [7]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)


In [8]:
# Initialize and train TPOT
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2, random_state=123)


In [9]:
tpot.fit(X_train, y_train)

# Evaluate performance on test data
accuracy = tpot.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.2f}")



Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6696015525546347

Generation 2 - Current best internal CV score: 0.6696015525546347

Generation 3 - Current best internal CV score: 0.6737999068173731

Generation 4 - Current best internal CV score: 0.6737999068173731

Generation 5 - Current best internal CV score: 0.6737999068173731

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.6500000000000001, min_samples_leaf=4, min_samples_split=13, n_estimators=100)
Model accuracy: 0.66


In [10]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 MB 4.1 MB/s eta 0:00:00


In [11]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpce_wsxqd
  JVM stdout: /tmp/tmpce_wsxqd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpce_wsxqd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_unknownUser_17jfcy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [13]:
# Load your data into H2O
h2o_df = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
# Set predictors and response column
predictors = ["Engine rpm", "Lub oil pressure", "Fuel pressure", "Coolant pressure", "lub oil temp", "Coolant temp"]
response = "Engine Condition"


In [15]:
# Split data
train, valid = h2o_df.split_frame(ratios=[0.8], seed=1234)

# Initialize AutoML
aml = H2OAutoML(max_models=5, seed=1234)
aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

# Get the leaderboard
leaderboard = aml.leaderboard
print(leaderboard)

AutoML progress: |
07:22:23.48: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
07:22:23.162: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

██
07:22:40.190: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


07:22:42.438: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███
07: